# **Programming Assessment \#4**

Names: \<please supply your names\>

More information on the assessment is found in our Canvas course.

# **Load Data**

*While you don't have to separate your code into blocks, it might be easier if you separated loading your data from actually implementation of your code. Consider placing all loading of data into the code block below.*

In [141]:
# The code is defining a function called `make_p` that takes a DataFrame `df` as input.
# lang must contain a column 'count' that contains number types
#load error data
import pandas as pd
%pip install editdistpy
from editdistpy import damerau_osa as ld
def make_p(df):
    total = df['count'].sum()
    inv = 1/total
    df['P'] = df['count']*inv
    print (df['P'])
    return df


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [142]:
x=''
with open("count_1edit.txt", "r") as f:
    x = f.read().splitlines()
gen = (dat.split('\t') for dat in x)

In [143]:
data = [(rec[0],int(rec[1]))for rec in gen]
err = pd.DataFrame.from_records(data, columns=['error', 'count'])
err = make_p(err)
err = err.set_index(['error'])
print(err)

0       0.023469
1       0.021908
2       0.019732
3       0.019169
4       0.014307
          ...   
1582    0.000026
1583    0.000026
1584    0.000026
1585    0.000026
1586    0.000026
Name: P, Length: 1587, dtype: float64
       count         P
error                 
e|i      917  0.023469
a|e      856  0.021908
i|e      771  0.019732
e|a      749  0.019169
a|i      559  0.014307
...      ...       ...
 |c        1  0.000026
 |a        1  0.000026
 |'        1  0.000026
 w|        1  0.000026
 t|t       1  0.000026

[1587 rows x 2 columns]


In [144]:
#loading corpus files
import nltk
nltk.download('gutenburg')
# nltk.corpus.gutenberg.fileids()

[nltk_data] Error loading gutenburg: Package 'gutenburg' not found in
[nltk_data]     index


In [149]:
from collections import Counter
# TODO: consolidate into a Finder class
from string import punctuation as punc, digits as dig
print("Extracting all documents from NLTK's Project Gutenberg Collection...")
all_words = Counter()

for filename in nltk.corpus.gutenberg.fileids():
  words = [word.lower() for word in nltk.corpus.gutenberg.words(filename) if word not in punc and word not in dig]
  all_words.update(words)

Extracting all documents from NLTK's Project Gutenberg Collection...


In [150]:
lang = pd.DataFrame.from_dict(all_words, orient='index').reset_index()
print(lang)

          index     0
0          emma   866
1            by  8512
2          jane   303
3        austen     3
4          1816     1
...         ...   ...
42287  endowing     1
42288   delving     1
42289  germinal     1
42290   blither     1
42291  ushering     1

[42292 rows x 2 columns]


In [151]:
lang = lang.rename(columns={'index':'word', 0:'count'})
lang = lang.set_index(['word'])

In [152]:
lang = make_p(lang)
print(lang)

word
emma        3.939490e-04
by          3.872164e-03
jane        1.378367e-04
austen      1.364719e-06
1816        4.549065e-07
                ...     
endowing    4.549065e-07
delving     4.549065e-07
germinal    4.549065e-07
blither     4.549065e-07
ushering    4.549065e-07
Name: P, Length: 42292, dtype: float64
          count             P
word                         
emma        866  3.939490e-04
by         8512  3.872164e-03
jane        303  1.378367e-04
austen        3  1.364719e-06
1816          1  4.549065e-07
...         ...           ...
endowing      1  4.549065e-07
delving       1  4.549065e-07
germinal      1  4.549065e-07
blither       1  4.549065e-07
ushering      1  4.549065e-07

[42292 rows x 2 columns]


# **Noisy Channel Model Implementation**

*Again, you don't have to follow this directly, but consider placing your implementation of the model in the code block below.*

In [153]:
def candidates(input, all_words):
      # 1 edit distance away from word
      if input in all_words: return [input]
      # candidates = [w for w in all_words if get_d(input,w) <3 ]

      candidates = None
      d = 1
      MAX = 90 #magic number set to the longest possible edit distance ig?
      while not candidates and d <= MAX:
            candidates = [w for w in all_words if ld.distance(input,w,d) > -1]
            d+=1

      return candidates
def spell_correct(candidate_list,tok):
    if tok in candidate_list: return tok
    n=10
    probable = lang.loc[candidate_list].sort_values(['P'], ascending=False).head(n)
    print(lang.loc[candidate_list])
      # maxI = 0
      # for i in range(len(candidate_list)):
      #       x = P(candidate_list[i], all_words)
      #       if x > maxX:
      #             maxX = x
      #             maxI = i
      # TODO: Add the error model. Kyle, can you please make an algo that finds the first error and query the probability?
    return probable.index

In [169]:
def get_err(input_word, candidate_word):
    print(candidate_word)
    if len(input_word) == len(candidate_word) and input_word != candidate_word:
        for i in range(len(input_word) - 1):
            if input_word[i] == candidate_word[i + 1] and input_word[i + 1] == candidate_word[i]:
                edit_type = "trans"
                edit ="|".join(( candidate_word[i:i+2]  ,input_word[i:i+2]))
                print(edit_type)
                return edit_type, edit
            if input_word[i] != candidate_word[i]:
                edit_type = "sub"
                edit ="|".join(( candidate_word[i]  ,input_word[i]))
                print(edit_type)
                return edit_type, edit

    
            

    elif len(input_word) < len(candidate_word):
        # Check for insertion
        edit_type = "ins"
        edit = ""
        i, j = 0, 0
        while i < len(input_word) and j < len(candidate_word):
            if input_word[i] != candidate_word[j]:
                edit += input_word[j-1] + "|" + input_word[j-1] + candidate_word[j]
                print(edit_type)
                return edit_type, edit
        
            i += 1
            j += 1


    elif len(input_word) > len(candidate_word):
        # Check for deletion
        edit_type = "del"
        edit = ""
        i, j = 0, 0
        while i < len(input_word) and j < len(candidate_word):
            if input_word[i] != candidate_word[j]:
                edit += input_word[i-1] + input_word[i] + "|" + candidate_word[i - 1]
                print(edit_type)
                return edit_type, edit
       
            i += 1
            j += 1
        if i == len(input_word):
            return edit_type, edit

    # Check for transposition
    
    print('roke')
    return None, None  # No valid edit type found

In [171]:
#main function
test_input = input()
c = candidates(test_input, all_words)
if test_input in c:
    print("Word is spelled correctly")
else:
    df = pd.DataFrame(columns=['Word', 'candidate', 'edit_type', 'edit', 'P(c)', 'P(w|c)', 'P(c) x P(w|c)'])
    #show all candidates
    i = 0
    for candidate in c:
        #get probability of word
        wordP = lang.at[ candidate,'P']

        edit_type, edit = get_err(test_input, candidate)
        #get probability of error
        try:
            editP = err.at[edit,'P']
         
        except:
            editP = 0
        temp = pd.DataFrame({'Word' : test_input, 'candidate' : candidate, 'edit_type' : edit_type, 'edit' : edit, 'P(c)' : wordP, 'P(w|c)' : editP, 'P(c) x P(w|c)' : (wordP * editP)}
                            , index=[i])
        df = pd.concat([df, temp])
        i += 1
    print(df)
    

 mohter


mother
trans
morter
sub
     Word candidate edit_type   edit      P(c)    P(w|c)  P(c) x P(w|c)
0  mohter    mother     trans  th|ht  0.000552  0.000819   4.519144e-07
1  mohter    morter       sub    r|h  0.000005  0.000102   5.122690e-10
